In [1]:
import pandas as pd
import numpy as np
import html5lib
from bs4 import BeautifulSoup
import requests 

In [2]:
c = pd.read_csv('worldcities.csv')

In [3]:
c['capital'] = c['capital'].fillna('')
c['admin_name'] = c['admin_name'].fillna('')
c['population'] = c['population'].fillna(0)
c['population'] = c['population'].astype('int32')
c = c.set_index('id')
c = c.drop(columns=["iso2", "capital", "city"])
c

,city_ascii,lat,lng,country,iso3,admin_name,population
id,,,,,,,
1392685764,Tokyo,35.6897,139.6922,Japan,JPN,Tōkyō,37732000
1360771077,Jakarta,-6.1750,106.8275,Indonesia,IDN,Jakarta,33756000
1356872604,Delhi,28.6100,77.2300,India,IND,Delhi,32226000
1156237133,Guangzhou,23.1300,113.2600,China,CHN,Guangdong,26940000
1356226629,Mumbai,19.0761,72.8775,India,IND,Mahārāshtra,24973000
...,...,...,...,...,...,...,...
1643985006,Numto,63.6667,71.3333,Russia,RUS,Khanty-Mansiyskiy Avtonomnyy Okrug-Yugra,10
1304217709,Nord,81.7166,-17.8000,Greenland,GRL,,10
1304206491,Timmiarmiut,62.5333,-42.2167,Greenland,GRL,Kujalleq,10


In [4]:
c = c[c['population'] > 200000]

In [5]:
# HTML PARSE FOR REGIONS

In [73]:
url = 'https://cloford.com/resources/codes/index.htm'
r = requests.get(url) 
soup = BeautifulSoup(r.content, 'html5lib')

In [74]:
table = pd.read_html(str(soup.find('table', {'class': 'outlinetable'})))
regions = table[0] #parses all tables and returns list, but we only have one.
regions

,Continent,Region,Country,Capital,FIPS,ISO (2),ISO (3),ISO (No),Internet,Note
0,Asia,South Asia,Afghanistan,Kabul,AF,AF,AFG,4.0,AF,NaN
1,Europe,South East Europe,Albania,Tirana,AL,AL,ALB,8.0,AL,NaN
2,Africa,Northern Africa,Algeria,Algiers,AG,DZ,DZA,12.0,DZ,NaN
3,Oceania,Pacific,American Samoa,Pago Pago,AQ,AS,ASM,16.0,AS,NaN
4,Europe,South West Europe,Andorra,Andorra la Vella,AN,AD,AND,20.0,AD,NaN
...,...,...,...,...,...,...,...,...,...,...
227,Oceania,Pacific,Western Samoa,Apia,WS,WS,WSM,882.0,WS,NaN
228,Asia,South West Asia,Yemen,Sanaa,YM,YE,YEM,887.0,YE,NaN
229,Africa,Central Africa,Zaire (Dem Rep of Congo),Kinshasa,CG,ZR,ZAR,180.0,ZR,8.0
230,Africa,Southern Africa,Zambia,Lusaka,ZA,ZM,ZWB,894.0,ZM,NaN


In [79]:
cols = regions[['Continent', 'Region', 'Country', 'ISO (3)']]
cols = cols.rename(columns={"ISO (3)": 'iso3'})
adds = [['Europe', 'South East Europe', 'Serbia', 'SRB'],
        ['Asia', 'East Asia', 'Hong Kong', 'HKG'],
        ['Africa', 'Central Africa', 'South Sudan', 'SSD'],
        ['Asia', 'East Asia', 'Macau', 'MAC'],
        ['Asia', 'South West Asia', 'Gaza Strip', 'XGZ'],
        ['Asia', 'South East Asia', 'Timor-Leste', 'TLS'],
        ['Asia', 'South West Asia', 'West Bank', 'XWB']
       ]

addsdf = pd.DataFrame(data=adds, columns=cols.columns)
cols = pd.concat([cols, addsdf])

In [80]:
cols = cols.set_index('iso3')
cols = cols.rename(index={'ZAR': 'COD', 'ROM': 'ROU', 'ZWB': 'ZMB'})

In [81]:
cols.head(3)

,Continent,Region,Country
iso3,,,
AFG,Asia,South Asia,Afghanistan
ALB,Europe,South East Europe,Albania
DZA,Africa,Northern Africa,Algeria


In [83]:
c['continent'] = np.zeros(len(c))
c['region'] = np.zeros(len(c))

for i, r in c.iterrows():
    if r['iso3'] in cols.index:
        c.loc[i, 'continent'] = cols.loc[r['iso3'], 'Continent']
        c.loc[i, 'region'] = cols.loc[r['iso3'], 'Region']
    else:
        r['continent'] = ''
        r['region'] = ''

c

C:\Users\BrandonKaminski\AppData\Local\Temp\ipykernel_39940\1436035958.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  c['continent'] = np.zeros(len(c))
C:\Users\BrandonKaminski\AppData\Local\Temp\ipykernel_39940\1436035958.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  c['region'] = np.zeros(len(c))


,city_ascii,lat,lng,country,iso3,admin_name,population,continent,region
id,,,,,,,,,
1392685764,Tokyo,35.6897,139.6922,Japan,JPN,Tōkyō,37732000,Asia,East Asia
1360771077,Jakarta,-6.1750,106.8275,Indonesia,IDN,Jakarta,33756000,Asia,South East Asia
1356872604,Delhi,28.6100,77.2300,India,IND,Delhi,32226000,Asia,South Asia
1156237133,Guangzhou,23.1300,113.2600,China,CHN,Guangdong,26940000,Asia,East Asia
1356226629,Mumbai,19.0761,72.8775,India,IND,Mahārāshtra,24973000,Asia,South Asia
...,...,...,...,...,...,...,...,...,...
1356019851,Morena,26.5000,78.0000,India,IND,Madhya Pradesh,200483,Asia,South Asia
1024903782,Soio,-6.1333,12.3667,Angola,AGO,Zaire,200920,Africa,Southern Africa
1862573240,Guacara,10.2333,-67.8833,Venezuela,VEN,Carabobo,200212,Americas,South America


In [85]:
len(c[c['continent'] == 0.0]) == 0

True

In [86]:
c.to_json('worldcities.json')

In [94]:
cs = list(c['country'])
csdict = {}
for co in set(cs):
    csdict[co] = cs.count(co)

sorted(csdict.items(), key=lambda x:x[1], reverse=True)

[('China', 623),
 ('India', 266),
 ('United States', 234),
 ('Brazil', 139),
 ('Japan', 134),
 ('Russia', 97),
 ('Mexico', 91),
 ('Philippines', 79),
 ('Turkey', 79),
 ('Indonesia', 77),
 ('Nigeria', 62),
 ('South Korea', 52),
 ('Vietnam', 51),
 ('Iran', 48),
 ('Germany', 40),
 ('United Kingdom', 40),
 ('Pakistan', 40),
 ('Colombia', 33),
 ('Ukraine', 32),
 ('Venezuela', 31),
 ('Bangladesh', 31),
 ('Spain', 30),
 ('Iraq', 29),
 ('Egypt', 28),
 ('Canada', 27),
 ('South Africa', 25),
 ('Argentina', 25),
 ('Angola', 24),
 ('Malaysia', 24),
 ('Saudi Arabia', 23),
 ('Congo (Kinshasa)', 19),
 ('Morocco', 19),
 ('Peru', 18),
 ("Côte d'Ivoire", 18),
 ('Sudan', 17),
 ('Chile', 17),
 ('Poland', 17),
 ('Italy', 16),
 ('Myanmar', 15),
 ('North Korea', 15),
 ('Somalia', 15),
 ('Kazakhstan', 14),
 ('Taiwan', 13),
 ('Cuba', 11),
 ('Australia', 11),
 ('Uzbekistan', 11),
 ('France', 11),
 ('Algeria', 10),
 ('Ethiopia', 10),
 ('Bolivia', 10),
 ('Romania', 9),
 ('Dominican Republic', 9),
 ('Netherlands',